# 🇫🇷 French Assistant - RAG-система для переводчика

**Архитектура:**
1. Document Loader → загрузка базы знаний
2. Text Splitter → разбиение на chunks с контролем семантики
3. Embeddings → мультиязычные эмбеддинги
4. Vector Store → ChromaDB для хранения и поиска
5. Retriever → MMR + Re-ranking для качественного поиска
6. LLM Chain → генерация ответов с грамматическим контролем
7. Safety Layer → фильтрация запросов и ответов

## 📦 1. Установка зависимостей

In [ ]:
!pip install -q langchain langchain-community langchain-core langchain-text-splitters
!pip install -q langchain-huggingface chromadb
!pip install -q transformers sentence-transformers
!pip install -q pyyaml accelerate bitsandbytes
!pip install -q huggingface_hub

## 🔑 1.1 Настройка HuggingFace токена (опционально)

Токен необходим для доступа к gated моделям (Llama, Saiga и др.)

Получить токен: https://huggingface.co/settings/tokens

In [ ]:
import os
from getpass import getpass

# Способ 1: Ввод токена вручную (безопасный ввод)
# Раскомментируйте строку ниже и введите токен
# HF_TOKEN = getpass("🔑 Введите HuggingFace токен: ")

# Способ 2: Использование Colab Secrets (рекомендуется для Colab)
try:
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    if HF_TOKEN:
        print("✅ Токен загружен из Colab Secrets")
except:
    HF_TOKEN = None

# Способ 3: Переменная окружения
if not HF_TOKEN:
    HF_TOKEN = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_TOKEN")
    if HF_TOKEN:
        print("✅ Токен загружен из переменной окружения")

# Устанавливаем токен для huggingface_hub
if HF_TOKEN:
    os.environ["HF_TOKEN"] = HF_TOKEN
    os.environ["HUGGINGFACE_TOKEN"] = HF_TOKEN
    print(f"✅ HuggingFace токен установлен (длина: {len(HF_TOKEN)} символов)")
else:
    print("⚠️ HuggingFace токен не найден")
    print("   Для работы с gated моделями раскомментируйте getpass выше")
    print("   или добавьте токен в Colab Secrets (ключ: HF_TOKEN)")

## 🤖 1.2 Загрузка модели Saiga LLaMA3 8B

Saiga — русскоязычная модель на базе LLaMA3, оптимизированная для диалогов.

**Требования:**
- GPU с минимум 16GB VRAM (T4, A100)
- HuggingFace токен (модель gated)

**Квантизация:** Используем 4-bit квантизацию для экономии памяти.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Проверка GPU
if not torch.cuda.is_available():
    print("⚠️ GPU не обнаружен! Saiga требует GPU для работы.")
    print("   В Colab: Runtime → Change runtime type → GPU")
    SAIGA_MODEL = None
    SAIGA_TOKENIZER = None
else:
    print(f"✅ GPU доступен: {torch.cuda.get_device_name(0)}")
    print(f"   Память: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    
    # Конфигурация модели
    MODEL_NAME = "IlyaGusev/saiga_llama3_8b"
    
    # 4-bit квантизация для экономии памяти
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    
    print(f"\n🔄 Загрузка модели {MODEL_NAME}...")
    print("   (Это может занять несколько минут при первом запуске)")
    
    # Загрузка токенизатора
    SAIGA_TOKENIZER = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        token=HF_TOKEN,
        trust_remote_code=True
    )
    
    # Загрузка модели с квантизацией
    SAIGA_MODEL = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        token=HF_TOKEN,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.float16,
    )
    
    print(f"✅ Модель {MODEL_NAME} загружена!")
    print(f"   Используемая память GPU: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

In [ ]:
class SaigaLLM:
    """
    Обёртка для модели Saiga для использования в RAG pipeline.
    
    Использует формат промптов Saiga (на базе ChatML).
    """
    
    def __init__(self, model, tokenizer, generation_config=None):
        self.model = model
        self.tokenizer = tokenizer
        self.generation_config = generation_config or {
            'max_new_tokens': 1024,
            'temperature': 0.3,
            'top_p': 0.9,
            'repetition_penalty': 1.1,
            'do_sample': True,
        }
    
    def _format_prompt(self, system_prompt: str, user_message: str) -> str:
        """Форматирует промпт в формате Saiga/ChatML."""
        # Saiga использует формат ChatML
        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>

{user_message}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
        return prompt
    
    def generate(self, system_prompt: str, user_message: str) -> str:
        """Генерирует ответ на основе системного промпта и сообщения пользователя."""
        if self.model is None:
            return "[Модель не загружена]"
        
        prompt = self._format_prompt(system_prompt, user_message)
        
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=self.generation_config['max_new_tokens'],
                temperature=self.generation_config['temperature'],
                top_p=self.generation_config['top_p'],
                repetition_penalty=self.generation_config['repetition_penalty'],
                do_sample=self.generation_config['do_sample'],
                pad_token_id=self.tokenizer.eos_token_id,
            )
        
        # Декодируем только новые токены
        response = self.tokenizer.decode(
            outputs[0][inputs['input_ids'].shape[1]:], 
            skip_special_tokens=True
        )
        
        return response.strip()
    
    def predict(self, prompt: str) -> str:
        """Совместимость с LangChain интерфейсом."""
        # Простая генерация без разделения на system/user
        return self.generate("", prompt)


# Создаём экземпляр LLM если модель загружена
if SAIGA_MODEL is not None and SAIGA_TOKENIZER is not None:
    SAIGA_LLM = SaigaLLM(SAIGA_MODEL, SAIGA_TOKENIZER)
    print("✅ SaigaLLM обёртка создана!")
else:
    SAIGA_LLM = None
    print("⚠️ SaigaLLM не создан (модель не загружена)")

## ⚙️ 2. Импорты и конфигурация

In [ ]:
import os
import re
import logging
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum
from collections import Counter
import hashlib
import yaml

# Проверка доступности GPU
import torch
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Используется устройство: {DEVICE}")

# LangChain imports (updated for langchain >= 0.2)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate

# HuggingFace Embeddings - с fallback для совместимости
try:
    from langchain_huggingface import HuggingFaceEmbeddings
except ImportError:
    from langchain_community.embeddings import HuggingFaceEmbeddings

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Импорты загружены успешно!")

In [ ]:
# Конфигурация (можно загрузить из YAML или задать здесь)
CONFIG = {
    'VECTOR_DB': {
        'persist_directory': './data/chroma_db',
        'collection_name': 'french_knowledge',
        'embeddings': {
            'model': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
            'device': DEVICE  # Автоопределение GPU/CPU
        }
    },
    'RAG_CONFIG': {
        'chunking': {
            'chunk_size': 500,
            'chunk_overlap': 100,
            'separators': ["\n\n", "\n", ".", " "]
        },
        'retrieval': {
            'k': 5,
            'fetch_k': 20,
            'lambda_mult': 0.7,
            'top_k_final': 3
        }
    },
    'SAFETY': {
        'input_filter': {
            'max_length': 2000
        }
    },
    'SYSTEM_PROMPT': '''Ты — профессиональный переводчик-ассистент, специализирующийся на русско-французском переводе.
Твои задачи:
- Помогать с переводом с русского на французский
- Объяснять грамматические правила французского языка
- Разъяснять идиомы и фразеологизмы
- Указывать на типичные ошибки и как их избежать

Всегда давай точные, обоснованные ответы с примерами.'''
}

print(f"✅ Конфигурация загружена! (device: {CONFIG['VECTOR_DB']['embeddings']['device']})")

## 🔧 3. Утилиты: Tracing и Safety

In [ ]:
@dataclass
class TraceEvent:
    """Событие трассировки для отладки и анализа."""
    timestamp: datetime
    event_type: str
    component: str
    input_data: Any
    output_data: Any
    metadata: Dict = field(default_factory=dict)
    duration_ms: float = 0.0


class TracingManager:
    """Менеджер трассировки для отслеживания всего pipeline."""
    
    def __init__(self):
        self.events: List[TraceEvent] = []
        self.session_id = hashlib.md5(str(datetime.now()).encode()).hexdigest()[:8]
    
    def log_event(self, event_type: str, component: str, 
                  input_data: Any, output_data: Any, 
                  metadata: Dict = None, duration_ms: float = 0.0):
        """Записывает событие трассировки."""
        event = TraceEvent(
            timestamp=datetime.now(),
            event_type=event_type,
            component=component,
            input_data=str(input_data)[:500],
            output_data=str(output_data)[:500],
            metadata=metadata or {},
            duration_ms=duration_ms
        )
        self.events.append(event)
        logger.debug(f"[TRACE][{component}] {event_type}: {str(input_data)[:100]}...")
    
    def get_trace_report(self) -> str:
        """Генерирует отчёт о трассировке."""
        report = [f"\n{'='*60}", f"TRACE REPORT - Session: {self.session_id}", f"{'='*60}"]
        for event in self.events:
            report.append(f"\n[{event.timestamp.strftime('%H:%M:%S.%f')[:-3]}] "
                         f"{event.component} -> {event.event_type}")
            report.append(f"  Input: {event.input_data[:100]}...")
            report.append(f"  Output: {event.output_data[:100]}...")
            if event.duration_ms > 0:
                report.append(f"  Duration: {event.duration_ms:.2f}ms")
        return "\n".join(report)


print("✅ TracingManager готов!")

In [ ]:
class SafetyFilter:
    """
    Фильтр безопасности для входных и выходных данных.
    
    Защита от:
    1. Prompt injection
    2. Off-topic запросов
    3. Потенциально вредного контента
    4. Галлюцинаций (проверка на основе retrieved контекста)
    """
    
    INJECTION_PATTERNS = [
        r"(?i)ignore\s+(all\s+)?(previous|above|prior)\s+(instructions?|prompts?)",
        r"(?i)forget\s+(everything|all|what)",
        r"(?i)you\s+are\s+now\s+a",
        r"(?i)new\s+instructions?:",
        r"(?i)system\s*prompt",
        r"(?i)jailbreak",
        r"(?i)bypass\s+(the\s+)?(filter|safety|security)",
        r"(?i)act\s+as\s+(if|though)",
        r"(?i)pretend\s+(to\s+be|you\s+are)",
        r"(?i)disregard\s+(your|the)\s+(rules|instructions)",
        r"(?i)override\s+(the\s+)?(system|safety)",
    ]
    
    TOPIC_KEYWORDS_RU = [
        'перевод', 'переведи', 'французск', 'франция', 'как сказать',
        'по-французски', 'грамматик', 'артикль', 'глагол', 'слово',
        'выражение', 'фраза', 'идиом', 'означает', 'перевести',
        'язык', 'произношение', 'accent', 'время глагол'
    ]
    
    TOPIC_KEYWORDS_FR = [
        'traduire', 'traduction', 'français', 'russe', 'grammaire',
        'verbe', 'article', 'expression', 'mot', 'phrase'
    ]
    
    def __init__(self, config: Dict):
        self.config = config
        self.max_length = config.get('max_length', 2000)
        self.blocked_patterns = [re.compile(p) for p in self.INJECTION_PATTERNS]
        logger.info(f"SafetyFilter initialized with {len(self.INJECTION_PATTERNS)} injection patterns")
    
    def check_injection(self, text: str) -> Tuple[bool, str]:
        """Проверяет текст на prompt injection."""
        for pattern in self.blocked_patterns:
            if pattern.search(text):
                return False, f"Обнаружена попытка injection: {pattern.pattern}"
        return True, ""
    
    def check_topic_relevance(self, text: str) -> Tuple[bool, float]:
        """Проверяет релевантность запроса теме французского языка."""
        text_lower = text.lower()
        
        ru_matches = sum(1 for kw in self.TOPIC_KEYWORDS_RU if kw in text_lower)
        fr_matches = sum(1 for kw in self.TOPIC_KEYWORDS_FR if kw in text_lower)
        
        score = (ru_matches + fr_matches) / min(10, len(text.split()))
        has_french_chars = bool(re.search(r'[éèêëàâäùûüôöîïç]', text_lower))
        
        is_relevant = ru_matches > 0 or fr_matches > 0 or has_french_chars or score > 0.1
        return is_relevant, min(score, 1.0)
    
    def check_length(self, text: str) -> Tuple[bool, str]:
        """Проверяет длину входного текста."""
        if len(text) > self.max_length:
            return False, f"Текст слишком длинный ({len(text)} > {self.max_length})"
        return True, ""
    
    def filter_input(self, text: str) -> Tuple[bool, str, Dict]:
        """Комплексная фильтрация входного запроса."""
        metadata = {'original_length': len(text), 'checks_passed': []}
        
        # 1. Проверка длины
        is_ok, msg = self.check_length(text)
        if not is_ok:
            return False, msg, metadata
        metadata['checks_passed'].append('length')
        
        # 2. Проверка на injection
        is_ok, msg = self.check_injection(text)
        if not is_ok:
            logger.warning(f"Injection detected: {text[:100]}...")
            return False, "Извините, я не могу обработать этот запрос.", metadata
        metadata['checks_passed'].append('injection')
        
        # 3. Проверка релевантности теме
        is_relevant, score = self.check_topic_relevance(text)
        metadata['topic_score'] = score
        
        if not is_relevant:
            return False, ("Извините, я специализируюсь только на переводе "
                          "с русского на французский и вопросах французского языка. "
                          "Пожалуйста, задайте вопрос по этой теме."), metadata
        metadata['checks_passed'].append('topic_relevance')
        
        return True, "", metadata
    
    def check_hallucination(self, response: str, context: str) -> Tuple[bool, float, List[str]]:
        """Проверяет ответ на возможные галлюцинации."""
        ungrounded_claims = []
        
        french_words_in_response = re.findall(
            r'\b[A-Za-zéèêëàâäùûüôöîïç]{3,}\b', 
            response
        )
        
        context_lower = context.lower()
        grounded_count = 0
        
        for word in french_words_in_response:
            if word.lower() in context_lower:
                grounded_count += 1
            elif len(word) > 5:
                ungrounded_claims.append(word)
        
        total_words = len(french_words_in_response)
        if total_words == 0:
            return True, 1.0, []
        
        grounding_score = grounded_count / total_words
        is_grounded = grounding_score > 0.3
        
        return is_grounded, grounding_score, ungrounded_claims[:5]


print("✅ SafetyFilter готов!")

## 🔍 4. Retrieval система

In [ ]:
class QueryExpander:
    """
    Расширитель запросов для улучшения поиска.
    
    Техники:
    1. Синонимы и переформулировки
    2. HyDE (Hypothetical Document Embeddings)
    3. Multi-query генерация
    """
    
    def __init__(self, llm=None):
        self.llm = llm
        self.synonyms = {
            'перевод': ['перевести', 'translation', 'traduire'],
            'глагол': ['verb', 'verbe', 'спряжение'],
            'артикль': ['article', 'определённый', 'неопределённый'],
            'время': ['tense', 'temps', 'présent', 'passé', 'futur'],
            'идиома': ['выражение', 'фразеологизм', 'idiome', 'expression'],
        }
    
    def expand_with_synonyms(self, query: str) -> List[str]:
        """Расширяет запрос синонимами."""
        expanded = [query]
        query_lower = query.lower()
        
        for term, syns in self.synonyms.items():
            if term in query_lower:
                for syn in syns:
                    expanded.append(query_lower.replace(term, syn))
        
        return list(set(expanded))[:4]
    
    def generate_hyde_document(self, query: str) -> str:
        """Генерирует гипотетический документ для HyDE."""
        if self.llm:
            prompt = f"""Напиши краткий информативный абзац, который бы отвечал на вопрос:
            "{query}"
            Ответ должен быть на русском языке и касаться французской грамматики или перевода."""
            try:
                return self.llm.predict(prompt)
            except Exception as e:
                logger.warning(f"HyDE generation failed: {e}")
        
        return f"Информация о французском языке по теме: {query}. Грамматические правила и примеры использования."
    
    def expand_query(self, query: str, use_hyde: bool = False) -> List[str]:
        """Комплексное расширение запроса."""
        queries = self.expand_with_synonyms(query)
        
        if use_hyde:
            hyde_doc = self.generate_hyde_document(query)
            queries.append(hyde_doc)
        
        logger.debug(f"Query expanded: {query} -> {len(queries)} variants")
        return queries


print("✅ QueryExpander готов!")

In [ ]:
class EnhancedRetriever:
    """
    Улучшенный ретривер с несколькими техниками:
    1. Multi-query retrieval
    2. MMR (Maximum Marginal Relevance)
    3. Re-ranking с CrossEncoder
    4. Contextual Compression
    """
    
    def __init__(self, vectorstore: Chroma, config: Dict, tracer: TracingManager):
        self.vectorstore = vectorstore
        self.config = config
        self.tracer = tracer
        
        self.base_retriever = vectorstore.as_retriever(
            search_type="mmr",
            search_kwargs={
                "k": config.get('k', 5),
                "fetch_k": config.get('fetch_k', 20),
                "lambda_mult": config.get('lambda_mult', 0.7)
            }
        )
        
        self.query_expander = QueryExpander()
        logger.info("EnhancedRetriever initialized with MMR search")
    
    def retrieve(self, query: str) -> List[Document]:
        """Выполняет улучшенный поиск документов."""
        import time
        start_time = time.time()
        
        # 1. Расширяем запрос
        expanded_queries = self.query_expander.expand_query(query, use_hyde=True)
        
        self.tracer.log_event(
            "query_expansion", "EnhancedRetriever",
            query, f"{len(expanded_queries)} variants",
            {"variants": expanded_queries}
        )
        
        # 2. Поиск по всем вариантам запроса
        all_docs = []
        seen_contents = set()
        
        for q in expanded_queries:
            docs = self.base_retriever.get_relevant_documents(q)
            for doc in docs:
                content_hash = hashlib.md5(doc.page_content.encode()).hexdigest()
                if content_hash not in seen_contents:
                    seen_contents.add(content_hash)
                    all_docs.append(doc)
        
        self.tracer.log_event(
            "multi_query_retrieval", "EnhancedRetriever",
            f"{len(expanded_queries)} queries", f"{len(all_docs)} unique docs"
        )
        
        # 3. Ранжирование по релевантности
        scored_docs = []
        query_terms = set(query.lower().split())
        
        for doc in all_docs:
            content_lower = doc.page_content.lower()
            score = sum(1 for term in query_terms if term in content_lower)
            length_score = min(len(doc.page_content) / 500, 1.0)
            final_score = score + length_score * 0.5
            scored_docs.append((final_score, doc))
        
        scored_docs.sort(key=lambda x: x[0], reverse=True)
        
        # 4. Возвращаем топ-k
        top_k = self.config.get('top_k_final', 3)
        result_docs = [doc for _, doc in scored_docs[:top_k]]
        
        duration = (time.time() - start_time) * 1000
        self.tracer.log_event(
            "retrieval_complete", "EnhancedRetriever",
            query, f"{len(result_docs)} final docs",
            {"scores": [s for s, _ in scored_docs[:top_k]]},
            duration
        )
        
        return result_docs


print("✅ EnhancedRetriever готов!")

## 🧠 5. RAG Enhancements (Anti-Hallucination)

In [ ]:
class RetrievalQuality(Enum):
    """Оценка качества retrieved документов."""
    EXCELLENT = "excellent"
    GOOD = "good"
    PARTIAL = "partial"
    POOR = "poor"
    AMBIGUOUS = "ambiguous"


@dataclass
class VerificationResult:
    """Результат верификации ответа."""
    is_verified: bool
    confidence: float
    issues: List[str]
    corrections: List[str]
    grounding_evidence: List[str]


print("✅ Базовые классы для RAG Enhancements готовы!")

In [ ]:
class ChainOfVerification:
    """
    Реализация Chain-of-Verification (CoVe).
    
    Процесс:
    1. Генерация черновика ответа
    2. Планирование проверочных вопросов
    3. Ответы на проверочные вопросы
    4. Генерация финального верифицированного ответа
    """
    
    def __init__(self, knowledge_base_content: str = ""):
        self.kb_content = knowledge_base_content
        self.verification_templates = {
            'factual': "Является ли утверждение '{claim}' фактически верным согласно базе знаний?",
            'consistency': "Согласуется ли '{claim}' с другими частями ответа?",
            'completeness': "Полностью ли раскрыт вопрос '{aspect}'?",
            'source': "Есть ли в базе знаний подтверждение для '{claim}'?"
        }
    
    def extract_claims(self, response: str) -> List[str]:
        """Извлекает ключевые утверждения из ответа."""
        claims = []
        
        translation_pattern = r'(?:перевод|французски|traduire)[:\s]+([^.!?\n]+)'
        claims.extend(re.findall(translation_pattern, response, re.IGNORECASE))
        
        grammar_pattern = r'(?:правило|используется|образуется|спрягается)[:\s]+([^.!?\n]+)'
        claims.extend(re.findall(grammar_pattern, response, re.IGNORECASE))
        
        example_pattern = r'(?:например|пример)[:\s]+([^.!?\n]+)'
        claims.extend(re.findall(example_pattern, response, re.IGNORECASE))
        
        return list(set(claims))[:10]
    
    def verify_claim(self, claim: str, context: str) -> Tuple[bool, float, str]:
        """Верифицирует отдельное утверждение."""
        claim_lower = claim.lower()
        context_lower = context.lower()
        
        claim_words = set(re.findall(r'\b\w{4,}\b', claim_lower))
        context_words = set(re.findall(r'\b\w{4,}\b', context_lower))
        
        overlap = claim_words & context_words
        
        if len(claim_words) == 0:
            return True, 1.0, "Empty claim"
        
        overlap_ratio = len(overlap) / len(claim_words)
        
        evidence = ""
        for word in overlap:
            sentences = re.split(r'[.!?]', context)
            for sent in sentences:
                if word in sent.lower():
                    evidence = sent.strip()
                    break
        
        is_verified = overlap_ratio > 0.3
        confidence = min(overlap_ratio * 1.5, 1.0)
        
        return is_verified, confidence, evidence
    
    def run_verification(self, response: str, context: str) -> VerificationResult:
        """Выполняет полную верификацию ответа."""
        claims = self.extract_claims(response)
        
        issues = []
        corrections = []
        grounding_evidence = []
        
        verified_count = 0
        total_confidence = 0.0
        
        for claim in claims:
            is_verified, confidence, evidence = self.verify_claim(claim, context)
            total_confidence += confidence
            
            if is_verified:
                verified_count += 1
                if evidence:
                    grounding_evidence.append(evidence[:100])
            else:
                issues.append(f"Не подтверждено: {claim[:50]}...")
                corrections.append(f"Рекомендуется проверить: {claim[:50]}...")
        
        overall_verified = verified_count / max(len(claims), 1) > 0.5
        avg_confidence = total_confidence / max(len(claims), 1)
        
        return VerificationResult(
            is_verified=overall_verified,
            confidence=avg_confidence,
            issues=issues,
            corrections=corrections,
            grounding_evidence=grounding_evidence[:5]
        )


print("✅ ChainOfVerification готов!")

In [ ]:
class SelfRAG:
    """
    Self-RAG: самооценка качества retrieval и генерации.
    
    Компоненты:
    1. Retrieval Token - нужен ли retrieval?
    2. Relevance Token - релевантен ли документ?
    3. Support Token - поддерживает ли документ ответ?
    4. Utility Token - полезен ли ответ?
    """
    
    def __init__(self):
        self.retrieval_triggers = {
            'high': ['перевод', 'как сказать', 'правило', 'грамматика', 'спряжение'],
            'low': ['привет', 'спасибо', 'пока', 'как дела']
        }
    
    def assess_retrieval_need(self, query: str) -> Tuple[bool, float]:
        """Оценивает, нужен ли retrieval для данного запроса."""
        query_lower = query.lower()
        
        high_triggers = sum(1 for t in self.retrieval_triggers['high'] if t in query_lower)
        low_triggers = sum(1 for t in self.retrieval_triggers['low'] if t in query_lower)
        
        if low_triggers > high_triggers:
            return False, 0.9
        elif high_triggers > 0:
            return True, min(0.6 + high_triggers * 0.1, 1.0)
        else:
            return True, 0.7
    
    def assess_relevance(self, query: str, document: str) -> Tuple[RetrievalQuality, float]:
        """Оценивает релевантность документа запросу."""
        query_words = set(query.lower().split())
        doc_words = set(document.lower().split())
        
        intersection = len(query_words & doc_words)
        union = len(query_words | doc_words)
        jaccard = intersection / max(union, 1)
        
        key_terms_found = sum(1 for w in query_words if len(w) > 4 and w in document.lower())
        
        if jaccard > 0.3 and key_terms_found >= 2:
            return RetrievalQuality.EXCELLENT, 0.9
        elif jaccard > 0.2 or key_terms_found >= 1:
            return RetrievalQuality.GOOD, 0.7
        elif jaccard > 0.1:
            return RetrievalQuality.PARTIAL, 0.5
        else:
            return RetrievalQuality.POOR, 0.3
    
    def assess_support(self, response: str, documents: List[str]) -> Tuple[bool, float]:
        """Оценивает, поддерживается ли ответ документами."""
        combined_docs = " ".join(documents).lower()
        response_lower = response.lower()
        
        response_words = set(re.findall(r'\b[a-zа-яéèêëàâäùûüôöîïç]{4,}\b', response_lower))
        supported = sum(1 for w in response_words if w in combined_docs)
        support_ratio = supported / max(len(response_words), 1)
        
        return support_ratio > 0.3, support_ratio
    
    def assess_utility(self, query: str, response: str) -> Tuple[bool, float]:
        """Оценивает полезность ответа для запроса."""
        if len(response) < 50:
            return False, 0.2
        
        has_structure = any(marker in response for marker in ['📝', '💡', '📚', '⚠️', '**'])
        
        query_topics = set(re.findall(r'\b\w{4,}\b', query.lower()))
        response_topics = set(re.findall(r'\b\w{4,}\b', response.lower()))
        topic_coverage = len(query_topics & response_topics) / max(len(query_topics), 1)
        
        utility_score = (0.3 if has_structure else 0) + topic_coverage * 0.7
        return utility_score > 0.4, utility_score


print("✅ SelfRAG готов!")

In [ ]:
class CorrectiveRAG:
    """
    CRAG - Corrective Retrieval Augmented Generation.
    
    Стратегия:
    1. Оценка качества retrieved документов
    2. Если качество низкое - коррекция через:
       - Дополнительный поиск
       - Переформулировка запроса
       - Использование fallback-знаний
    """
    
    def __init__(self):
        self.self_rag = SelfRAG()
        self.cove = ChainOfVerification()
        
        self.fallback_knowledge = {
            'articles': "Во французском языке 3 типа артиклей: определённые (le, la, les), "
                       "неопределённые (un, une, des) и частичные (du, de la, de l').",
            'verbs': "Французские глаголы делятся на 3 группы: -er (1-я), -ir с -issons (2-я) "
                    "и неправильные (3-я). Aller - исключение из 1-й группы.",
            'tenses': "Основные времена: présent (настоящее), passé composé (прошедшее составное), "
                     "imparfait (незавершенное), futur simple (простое будущее)."
        }
    
    def evaluate_retrieval_quality(self, query: str, documents: List[str]) -> RetrievalQuality:
        """Оценивает общее качество retrieved документов."""
        if not documents:
            return RetrievalQuality.POOR
        
        qualities = []
        for doc in documents:
            quality, _ = self.self_rag.assess_relevance(query, doc)
            qualities.append(quality)
        
        quality_order = [
            RetrievalQuality.EXCELLENT,
            RetrievalQuality.GOOD,
            RetrievalQuality.PARTIAL,
            RetrievalQuality.POOR
        ]
        
        for q in quality_order:
            if q in qualities[:3]:
                return q
        
        return RetrievalQuality.POOR
    
    def get_correction_strategy(self, quality: RetrievalQuality) -> str:
        """Определяет стратегию коррекции на основе качества."""
        strategies = {
            RetrievalQuality.EXCELLENT: "none",
            RetrievalQuality.GOOD: "supplement",
            RetrievalQuality.PARTIAL: "refine",
            RetrievalQuality.POOR: "fallback",
            RetrievalQuality.AMBIGUOUS: "clarify"
        }
        return strategies.get(quality, "fallback")
    
    def apply_correction(self, query: str, documents: List[str], strategy: str) -> Tuple[List[str], str]:
        """Применяет стратегию коррекции."""
        if strategy == "none":
            return documents, ""
        
        elif strategy == "supplement":
            note = "Добавлена базовая информация для полноты ответа."
            query_lower = query.lower()
            
            for topic, knowledge in self.fallback_knowledge.items():
                if topic in query_lower or any(kw in query_lower for kw in topic.split()):
                    documents.append(f"[Базовые знания] {knowledge}")
                    break
            
            return documents, note
        
        elif strategy == "refine":
            note = "Качество контекста ограничено. Рекомендуется уточнить вопрос."
            return documents, note
        
        elif strategy == "fallback":
            note = "⚠️ Релевантная информация не найдена. Используются базовые знания."
            fallback_docs = [f"[Базовые знания] {k}" for k in self.fallback_knowledge.values()]
            return fallback_docs, note
        
        elif strategy == "clarify":
            note = "Найдена противоречивая информация. Требуется уточнение вопроса."
            return documents, note
        
        return documents, ""
    
    def correct(self, query: str, documents: List[str]) -> Tuple[List[str], Dict[str, Any]]:
        """Основной метод коррекции."""
        quality = self.evaluate_retrieval_quality(query, documents)
        strategy = self.get_correction_strategy(quality)
        corrected_docs, note = self.apply_correction(query, documents, strategy)
        
        metadata = {
            'original_quality': quality.value,
            'strategy': strategy,
            'correction_applied': strategy != "none",
            'note': note,
            'original_doc_count': len(documents),
            'corrected_doc_count': len(corrected_docs)
        }
        
        logger.info(f"CRAG correction: {quality.value} -> {strategy}")
        return corrected_docs, metadata


print("✅ CorrectiveRAG готов!")

In [ ]:
class HallucinationDetector:
    """
    Детектор галлюцинаций на основе нескольких методов.
    
    Методы:
    1. Lexical Grounding - лексическое заземление
    2. Semantic Consistency - семантическая согласованность
    3. Factual Verification - фактологическая проверка
    4. Confidence Calibration - калибровка уверенности
    """
    
    def __init__(self):
        self.cove = ChainOfVerification()
        self.confidence_markers = {
            'high': ['всегда', 'никогда', 'точно', 'безусловно', '100%', 'гарантированно'],
            'medium': ['обычно', 'как правило', 'чаще всего', 'в большинстве'],
            'low': ['возможно', 'вероятно', 'может быть', 'иногда']
        }
    
    def check_lexical_grounding(self, response: str, context: str) -> Tuple[float, List[str]]:
        """Проверяет лексическое заземление ответа на контекст."""
        french_terms = re.findall(r'\b[a-zéèêëàâäùûüôöîïç]{3,}\b', response.lower())
        technical_terms = re.findall(r'\b(?:артикль|глагол|спряжение|время|наклонение)\w*', 
                                     response.lower())
        
        all_terms = set(french_terms + technical_terms)
        context_lower = context.lower()
        
        ungrounded = []
        grounded_count = 0
        
        for term in all_terms:
            if term in context_lower:
                grounded_count += 1
            elif len(term) > 4:
                ungrounded.append(term)
        
        score = grounded_count / max(len(all_terms), 1)
        return score, ungrounded[:10]
    
    def check_semantic_consistency(self, response: str) -> Tuple[bool, List[str]]:
        """Проверяет внутреннюю согласованность ответа."""
        inconsistencies = []
        sentences = re.split(r'[.!?]', response)
        
        negation_pairs = [
            ('всегда', 'никогда'),
            ('можно', 'нельзя'),
            ('правильно', 'неправильно'),
            ('мужской род', 'женский род'),
        ]
        
        for sent1 in sentences:
            for sent2 in sentences:
                if sent1 == sent2:
                    continue
                for pos, neg in negation_pairs:
                    if pos in sent1.lower() and neg in sent2.lower():
                        words1 = set(sent1.lower().split())
                        words2 = set(sent2.lower().split())
                        if len(words1 & words2) > 3:
                            inconsistencies.append(f"Возможное противоречие: {pos} vs {neg}")
        
        return len(inconsistencies) == 0, inconsistencies
    
    def check_confidence_calibration(self, response: str) -> Tuple[str, List[str]]:
        """Проверяет калибровку уверенности в ответе."""
        response_lower = response.lower()
        overconfident = []
        
        for marker in self.confidence_markers['high']:
            if marker in response_lower:
                for sent in re.split(r'[.!?]', response):
                    if marker in sent.lower():
                        overconfident.append(sent.strip())
        
        if len(overconfident) > 2:
            return 'overconfident', overconfident
        elif any(m in response_lower for m in self.confidence_markers['low']):
            return 'calibrated', []
        else:
            return 'neutral', []
    
    def detect(self, response: str, context: str) -> Dict[str, Any]:
        """Выполняет полную детекцию галлюцинаций."""
        results = {
            'has_hallucinations': False,
            'confidence': 1.0,
            'issues': [],
            'recommendations': []
        }
        
        # 1. Лексическое заземление
        grounding_score, ungrounded = self.check_lexical_grounding(response, context)
        results['grounding_score'] = grounding_score
        results['ungrounded_terms'] = ungrounded
        
        if grounding_score < 0.3:
            results['issues'].append("Низкое лексическое заземление на контекст")
            results['recommendations'].append("Добавьте ссылки на источники")
            results['has_hallucinations'] = True
        
        # 2. Семантическая согласованность
        is_consistent, inconsistencies = self.check_semantic_consistency(response)
        results['is_consistent'] = is_consistent
        results['inconsistencies'] = inconsistencies
        
        if not is_consistent:
            results['issues'].append("Обнаружены внутренние противоречия")
            results['recommendations'].append("Проверьте согласованность утверждений")
            results['has_hallucinations'] = True
        
        # 3. Калибровка уверенности
        conf_level, overconfident = self.check_confidence_calibration(response)
        results['confidence_level'] = conf_level
        results['overconfident_claims'] = overconfident
        
        if conf_level == 'overconfident':
            results['issues'].append("Обнаружены сверхуверенные утверждения")
            results['recommendations'].append("Добавьте модификаторы неопределённости")
        
        # 4. Общая оценка уверенности
        results['confidence'] = (
            grounding_score * 0.4 +
            (1.0 if is_consistent else 0.5) * 0.3 +
            (0.7 if conf_level == 'calibrated' else 1.0 if conf_level == 'neutral' else 0.5) * 0.3
        )
        
        # 5. Верификация через CoVe
        verification = self.cove.run_verification(response, context)
        results['verification'] = {
            'is_verified': verification.is_verified,
            'confidence': verification.confidence,
            'issues': verification.issues[:3],
            'evidence': verification.grounding_evidence[:3]
        }
        
        if not verification.is_verified:
            results['has_hallucinations'] = True
            results['confidence'] *= 0.7
        
        logger.info(f"Hallucination detection: has_hallucinations={results['has_hallucinations']}, "
                   f"confidence={results['confidence']:.2f}")
        
        return results


print("✅ HallucinationDetector готов!")

## 🤖 6. FrenchAssistant - основной класс

In [ ]:
class FrenchAssistant:
    """
    Главный класс нейро-сотрудника — переводчика-ассистента.
    
    Функционал:
    - Перевод с русского на французский
    - Объяснение грамматики
    - Работа с идиомами и фразеологизмами
    - Помощь с типичными ошибками
    
    Использует модель Saiga LLaMA3 8B для генерации ответов.
    """
    
    def __init__(self, config: Dict = None, llm: SaigaLLM = None):
        """Инициализация ассистента."""
        logger.info("Initializing FrenchAssistant...")
        
        self.config = config or CONFIG
        
        # Инициализация компонентов
        self.tracer = TracingManager()
        self.safety_filter = SafetyFilter(self.config.get('SAFETY', {}).get('input_filter', {}))
        
        # Эмбеддинги
        self.embeddings = self._init_embeddings()
        
        # Векторное хранилище
        self.vectorstore = self._init_vectorstore()
        
        # Retriever
        self.retriever = EnhancedRetriever(
            self.vectorstore,
            self.config.get('RAG_CONFIG', {}).get('retrieval', {}),
            self.tracer
        )
        
        # LLM - используем Saiga или fallback на шаблоны
        self.llm = llm or SAIGA_LLM
        if self.llm:
            logger.info("✅ LLM (Saiga) подключена!")
        else:
            logger.warning("⚠️ LLM не загружена, используются шаблонные ответы")
        
        # Промпт
        self.prompt_template = self._create_prompt_template()
        
        logger.info("FrenchAssistant initialized successfully!")
    
    def _init_embeddings(self) -> HuggingFaceEmbeddings:
        """Инициализирует мультиязычные эмбеддинги."""
        embed_config = self.config.get('VECTOR_DB', {}).get('embeddings', {})
        model_name = embed_config.get(
            'model', 
            'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
        )
        
        logger.info(f"Loading embeddings model: {model_name}")
        
        return HuggingFaceEmbeddings(
            model_name=model_name,
            model_kwargs={'device': embed_config.get('device', 'cpu')},
            encode_kwargs={'normalize_embeddings': True}
        )
    
    def _init_vectorstore(self) -> Chroma:
        """Инициализирует или загружает векторное хранилище."""
        db_config = self.config.get('VECTOR_DB', {})
        persist_dir = db_config.get('persist_directory', './data/chroma_db')
        collection_name = db_config.get('collection_name', 'french_knowledge')
        
        if os.path.exists(persist_dir):
            logger.info(f"Loading existing vectorstore from {persist_dir}")
            return Chroma(
                persist_directory=persist_dir,
                embedding_function=self.embeddings,
                collection_name=collection_name
            )
        
        # Создаём новую базу с демо-данными
        logger.info("Creating new vectorstore with demo data...")
        documents = self._create_demo_documents()
        
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings,
            persist_directory=persist_dir,
            collection_name=collection_name
        )
        
        logger.info(f"Vectorstore created with {len(documents)} documents")
        return vectorstore
    
    def _create_demo_documents(self) -> List[Document]:
        """Создаёт демо-документы для базы знаний."""
        demo_content = [
            # Артикли
            {
                'content': '''# Артикли во французском языке

## Определённые артикли (articles définis)
- le - мужской род единственного числа (le livre - книга)
- la - женский род единственного числа (la table - стол)
- l' - перед гласной или h немым (l'eau - вода, l'homme - человек)
- les - множественное число обоих родов (les livres - книги)

## Неопределённые артикли (articles indéfinis)
- un - мужской род (un chat - кот)
- une - женский род (une maison - дом)
- des - множественное число (des fleurs - цветы)

## Частичные артикли (articles partitifs)
- du - мужской род (du pain - хлеб)
- de la - женский род (de la viande - мясо)
- de l' - перед гласной (de l'eau - воды)
''',
                'metadata': {'topic': 'grammar', 'subtopic': 'articles', 'difficulty': 'beginner'}
            },
            # Глаголы 1-й группы
            {
                'content': '''# Глаголы 1-й группы (-er)

Глаголы на -er составляют около 90% всех французских глаголов.

## Спряжение в présent (настоящее время)
Пример: parler (говорить)
- je parle - я говорю
- tu parles - ты говоришь
- il/elle parle - он/она говорит
- nous parlons - мы говорим
- vous parlez - вы говорите
- ils/elles parlent - они говорят

## Важные исключения
Глагол aller (идти) - единственный глагол на -er, который относится к 3-й группе!
- je vais, tu vas, il va, nous allons, vous allez, ils vont
''',
                'metadata': {'topic': 'grammar', 'subtopic': 'verbs', 'difficulty': 'beginner'}
            },
            # Passé composé
            {
                'content': '''# Passé composé (прошедшее составное время)

## Образование
avoir/être в présent + participe passé

## С avoir (большинство глаголов)
- J'ai mangé - Я поел
- Tu as parlé - Ты поговорил
- Elle a fini - Она закончила

## С être (глаголы движения и возвратные)
Глаголы с être: aller, venir, arriver, partir, entrer, sortir, monter, descendre, 
naître, mourir, rester, tomber, devenir, revenir, rentrer, retourner, passer (по)

- Je suis allé(e) - Я пошёл/пошла
- Elle est partie - Она ушла
- Nous sommes arrivés - Мы прибыли

## Согласование
С être причастие согласуется с подлежащим в роде и числе.
''',
                'metadata': {'topic': 'grammar', 'subtopic': 'tenses', 'difficulty': 'intermediate'}
            },
            # Идиомы
            {
                'content': '''# Французские идиомы и выражения

## Погода и природа
- Il pleut des cordes - Льёт как из ведра (букв. "дождь верёвками")
- Avoir le cafard - Хандрить, быть в депрессии (букв. "иметь таракана")

## Еда
- Avoir la banane - Широко улыбаться (букв. "иметь банан")
- Raconter des salades - Врать, рассказывать небылицы (букв. "рассказывать салаты")
- C'est la fin des haricots - Это конец! (букв. "конец фасоли")

## Животные
- Poser un lapin - Не прийти на свидание (букв. "посадить кролика")
- Avoir d'autres chats à fouetter - Иметь дела поважнее (букв. "иметь других кошек для битья")
- Quand les poules auront des dents - Когда рак на горе свистнет (букв. "когда у кур будут зубы")
''',
                'metadata': {'topic': 'idioms', 'subtopic': 'expressions', 'difficulty': 'intermediate'}
            },
            # Типичные ошибки
            {
                'content': '''# Типичные ошибки русскоговорящих

## Ложные друзья переводчика
- magazine ≠ магазин (= журнал). Магазин = le magasin
- cabinet ≠ кабинет (= шкаф, туалет). Кабинет = le bureau
- sympathique ≠ симпатичный (= приятный). Симпатичный = joli, mignon
- accurate ≠ аккуратный (= точный). Аккуратный = soigné, ordonné

## Произношение
- Не произносится конечная согласная: Paris [paʁi], не [paʁis]
- Liaison (связывание): les amis [lezami]
- Носовые звуки: on [ɔ̃], an [ɑ̃], in [ɛ̃]

## Грамматика
- Двойное отрицание: Je NE sais PAS (не только ne, но и pas!)
- Порядок слов в вопросе: Est-ce que tu viens? или Viens-tu?
''',
                'metadata': {'topic': 'translation', 'subtopic': 'common_mistakes', 'difficulty': 'beginner'}
            }
        ]
        
        documents = []
        splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=100,
            separators=["\n\n", "\n", ".", " "]
        )
        
        for item in demo_content:
            doc = Document(page_content=item['content'], metadata=item['metadata'])
            chunks = splitter.split_documents([doc])
            documents.extend(chunks)
        
        return documents
    
    def _create_prompt_template(self) -> PromptTemplate:
        """Создаёт шаблон промпта."""
        template = (
            "{system_prompt}\n\n"
            "## Контекст из базы знаний:\n"
            "{context}\n\n"
            "## Вопрос пользователя:\n"
            "{question}\n\n"
            "## Твой ответ:\n"
        )
        
        return PromptTemplate(
            input_variables=["system_prompt", "context", "question"],
            template=template
        )
    
    def _generate_response(self, query: str, context: str) -> str:
        """Генерирует ответ на основе контекста с использованием Saiga LLM."""
        # Если LLM не загружена - используем шаблонный ответ
        if self.llm is None:
            return self._generate_template_response(query, context)
        
        # Формируем сообщение для LLM
        system_prompt = self.config.get('SYSTEM_PROMPT', '')
        user_message = f"""Контекст из базы знаний:
{context}

Вопрос пользователя:
{query}

Дай подробный и полезный ответ на основе контекста."""
        
        try:
            response = self.llm.generate(system_prompt, user_message)
            return response
        except Exception as e:
            logger.error(f"LLM generation error: {e}")
            return self._generate_template_response(query, context)
    
    def _generate_template_response(self, query: str, context: str) -> str:
        """Генерирует шаблонный ответ (fallback без LLM)."""
        query_lower = query.lower()
        
        if 'перевед' in query_lower or 'перевод' in query_lower:
            return (
                f"📝 **Запрос на перевод**\n\n"
                f"💡 **Релевантная информация из базы знаний:**\n"
                f"{context[:600]}\n\n"
                f"⚠️ **Примечание:** Для точного перевода рекомендуется указать контекст использования."
            )
        
        elif 'как сказать' in query_lower:
            return (
                f"📝 **Ваш вопрос:** {query}\n\n"
                f"📚 **Информация из базы знаний:**\n"
                f"{context[:600]}\n\n"
                f"💡 **Совет:** Обратите внимание на контекст и регистр речи."
            )
        
        elif 'грамматик' in query_lower or 'правил' in query_lower:
            return (
                f"📖 **Грамматический вопрос:** {query}\n\n"
                f"📚 **Информация из базы знаний:**\n"
                f"{context[:700]}\n\n"
                f"💡 **Комментарий:** Обратите внимание на примеры использования."
            )
        
        elif 'идиом' in query_lower or 'выражен' in query_lower:
            return (
                f"🗣️ **Вопрос об идиомах:** {query}\n\n"
                f"📚 **Информация из базы знаний:**\n"
                f"{context[:700]}\n\n"
                f"💡 **Совет:** Идиомы часто не переводятся буквально."
            )
        
        else:
            return (
                f"📝 **Ваш вопрос:** {query}\n\n"
                f"📚 **Релевантная информация:**\n"
                f"{context[:700]}\n\n"
                f"💡 **Примечание:** Уточните вопрос для более детального ответа."
            )
    
    def process_query(self, query: str) -> Dict[str, Any]:
        """Обрабатывает запрос пользователя."""
        import time
        start_time = time.time()
        
        result = {
            'query': query,
            'response': '',
            'sources': [],
            'metadata': {},
            'trace': None,
            'is_safe': True,
            'error': None,
            'used_llm': self.llm is not None
        }
        
        self.tracer.log_event("query_received", "FrenchAssistant", query, "processing")
        
        # 1. Safety check
        is_safe, error_msg, safety_meta = self.safety_filter.filter_input(query)
        result['metadata']['safety'] = safety_meta
        
        if not is_safe:
            result['is_safe'] = False
            result['response'] = error_msg
            result['error'] = error_msg
            self.tracer.log_event("safety_blocked", "SafetyFilter", query, error_msg)
            return result
        
        self.tracer.log_event("safety_passed", "SafetyFilter", query, "safe")
        
        # 2. Retrieval
        try:
            docs = self.retriever.retrieve(query)
            result['sources'] = [
                {
                    'content': doc.page_content[:200] + "...",
                    'metadata': doc.metadata
                }
                for doc in docs
            ]
            
            context = "\n\n---\n\n".join([doc.page_content for doc in docs])
            
            self.tracer.log_event(
                "retrieval_complete", "EnhancedRetriever",
                query, f"{len(docs)} documents retrieved"
            )
            
        except Exception as e:
            logger.error(f"Retrieval error: {e}")
            result['error'] = f"Ошибка поиска: {str(e)}"
            result['response'] = "Извините, произошла ошибка при поиске информации."
            return result
        
        # 3. Response generation
        try:
            response = self._generate_response(query, context)
            result['response'] = response
            
            llm_name = "Saiga LLM" if self.llm else "Template"
            self.tracer.log_event(
                "response_generated", llm_name,
                query[:50] + "...", response[:100] + "..."
            )
            
        except Exception as e:
            logger.error(f"Generation error: {e}")
            result['error'] = f"Ошибка генерации: {str(e)}"
            result['response'] = "Извините, произошла ошибка при генерации ответа."
            return result
        
        # 4. Hallucination check
        is_grounded, score, ungrounded = self.safety_filter.check_hallucination(response, context)
        result['metadata']['grounding'] = {
            'is_grounded': is_grounded,
            'score': score,
            'ungrounded_claims': ungrounded
        }
        
        if not is_grounded:
            logger.warning(f"Potential hallucination detected: {ungrounded}")
            result['response'] += "\n\n⚠️ *Некоторые части ответа могут требовать дополнительной проверки.*"
        
        # Финализация
        duration = (time.time() - start_time) * 1000
        result['metadata']['total_duration_ms'] = duration
        result['trace'] = self.tracer.get_trace_report()
        
        self.tracer.log_event(
            "query_complete", "FrenchAssistant",
            query[:50] + "...", f"Response generated in {duration:.2f}ms"
        )
        
        return result
    
    def get_statistics(self) -> Dict:
        """Возвращает статистику работы ассистента."""
        return {
            'total_events': len(self.tracer.events),
            'collection_count': self.vectorstore._collection.count(),
            'llm_loaded': self.llm is not None,
            'llm_model': 'IlyaGusev/saiga_llama3_8b' if self.llm else 'None (template mode)',
            'config': {
                'embedding_model': self.config.get('VECTOR_DB', {}).get('embeddings', {}).get('model'),
                'chunk_size': self.config.get('RAG_CONFIG', {}).get('chunking', {}).get('chunk_size'),
                'retrieval_k': self.config.get('RAG_CONFIG', {}).get('retrieval', {}).get('k')
            }
        }


print("✅ FrenchAssistant готов (с поддержкой Saiga LLM)!")

## 🚀 7. Демо и тестирование

In [ ]:
# Инициализация ассистента
print("🚀 Инициализация French Assistant...")
assistant = FrenchAssistant()
print("\n✅ Ассистент готов к работе!")

In [ ]:
# Статистика
stats = assistant.get_statistics()
print("📊 Статистика:")
for key, value in stats.items():
    print(f"  {key}: {value}")

In [ ]:
# Тест 1: Грамматический вопрос
query = "Как спрягаются глаголы первой группы во французском?"
result = assistant.process_query(query)

print(f"❓ Вопрос: {query}")
print(f"\n🤖 Ответ:\n{result['response']}")
print(f"\n📚 Источников найдено: {len(result['sources'])}")

In [ ]:
# Тест 2: Вопрос об артиклях
query = "Расскажи про артикли во французском языке"
result = assistant.process_query(query)

print(f"❓ Вопрос: {query}")
print(f"\n🤖 Ответ:\n{result['response']}")

In [ ]:
# Тест 3: Идиомы
query = "Какие есть интересные французские идиомы?"
result = assistant.process_query(query)

print(f"❓ Вопрос: {query}")
print(f"\n🤖 Ответ:\n{result['response']}")

In [ ]:
# Тест 4: Проверка safety filter (off-topic)
query = "Какая погода сегодня?"
result = assistant.process_query(query)

print(f"❓ Вопрос: {query}")
print(f"\n🤖 Ответ:\n{result['response']}")
print(f"\n🛡️ Safety: {result['is_safe']}")

In [ ]:
# Интерактивный режим
# ⚠️ Раскомментируйте код ниже для интерактивного общения с ассистентом
# В режиме "Run all" эта ячейка пропускается

"""
print("="*60)
print("🇫🇷 French Assistant - Интерактивный режим")
print("="*60)
print("Введите 'exit' для выхода\n")

while True:
    query = input("👤 Вы: ").strip()
    
    if query.lower() == 'exit':
        print("\nAu revoir! 👋")
        break
    
    if not query:
        continue
    
    result = assistant.process_query(query)
    print(f"\n🤖 Ассистент:\n{result['response']}\n")
    print("-"*40)
"""

print("💡 Интерактивный режим закомментирован для совместимости с 'Run all'")
print("   Раскомментируйте код выше для интерактивного общения")